Header Files

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

Node definition

In [ ]:
class GRU_Node(nn.Module):
    def __init__(self, input_size, hidden_size):
        """
        Inputs:
            input_size     - Dimensionality of the input of this RNN.
            hidden_size    - Dimensionality of the hidden state.
            output_size    - Dimensionality of the output of this RNN.
        """

        super(GRU_Node, self).__init__()
        self.hidden_size = hidden_size
        
        self.Wz = nn.Linear(input_size,hidden_size)
        self.Uz = nn.Linear(hidden_size,hidden_size,bias = False)

        self.Wr = nn.Linear(input_size,hidden_size)
        self.Ur = nn.Linear(hidden_size,hidden_size,bias = False)

        self.Wh = nn.Linear(input_size,hidden_size)
        self.Ur = nn.Linear(hidden_size,hidden_size,bias = False)

    def forward(self,x,h_prev):
        # Npdate
        zt = torch.sigmoid(self.Wz(x) + self.Uz(h_prev))
        # reset
        rt = torch.tanh(self.Wh(x) + self.Uh(rt*h_prev))
        # cand. h
        h_tilde = torch.tanh(self.Wh(x) + self.Uh(rt*h_prev))
        #ht
        ht = (1-zt)*h_prev + zt*h_tilde
        
        return ht

Layer Definition using Node, with final FC layer

In [6]:
class GRULayer(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(GRULayer,self).__init__()
        self.hidden_size = hidden_size
        self.gru_node = GRU_Node(input_size, hidden_size)
        # final fully connected layer
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        #x_shape = (batch_size,seqeunce_length(no. of data you look back on),input_size(or feature size))
        batch_size, seq_len, _ = x.size()
        h = torch.zeros(batch_size,self.hidden_size)

        h_vals = []
        for t in range(seq_len):
            xt = x[:,t,:]
            h = self.gru_node(xt,h)
            h_vals.append(h.unsqeeze(1))
            output = self.fc(h)

        h_vals = torch.cat(h_vals, dim=1)
        return h_vals, h, output

Training Method definition

In [ ]:
def train_n_validate(model,x_train, y_train, x_val,y_val, num_epochs, optimizer,loss_criterion,val_flag=0):
    train_loss = []
    val_loss = []

    for epochs in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        y_pred = model(x_train)
        train_loss = loss_criterion(y_pred,y_train)
        train_loss.backward()
        optimizer.step()
        train_loss.append(train_loss.item())

        print(f"Epoch [{epochs+1}/{num_epochs}] - Train Loss: {train_loss.item():.4f}", end='')

        if val_flag == 1:
            model.eval()
            with torch.nograd():
                y_val_pred = model(x_val)
                val_loss = loss_criterion(y_val_pred,y_val)
                val_loss.append(val_loss.item())    
    
    return y_pred, train_loss, val_loss


Plotting functions

In [11]:
def plot(x,y_act,y_pred):
    plt.figure(figsize=(8, 4))
    plt.plot(y_act.numpy(), label='True Capacity')
    plt.plot(y_pred.numpy(), label='Predicted Capacity')
    plt.title('Validation: True vs Predicted Battery Capacity')
    plt.xlabel('Sample Index')
    plt.ylabel('Capacity')
    plt.legend()
    plt.grid(True)
    plt.show()